<a href="https://colab.research.google.com/github/gustavoamora/patter_recognition_and_machine_learningning/blob/main/gustavo_aula10a_feature_selection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [28]:
from sklearn.datasets import fetch_covtype
X, y = fetch_covtype(return_X_y=True)
X.shape, y.shape

((581012, 54), (581012,))

In [29]:
from sklearn.model_selection import cross_validate
from sklearn.linear_model import RidgeClassifier
import numpy as np

modelo = RidgeClassifier()
scores = cross_validate(modelo, X, y)
scores, np.mean(scores['test_score'])

({'fit_time': array([1.04676032, 1.14223933, 0.9960804 , 1.27979732, 1.61488461]),
  'score_time': array([0.04362798, 0.04034114, 0.04267406, 0.06844759, 0.1014607 ]),
  'test_score': array([0.58947704, 0.76359474, 0.62652106, 0.67569405, 0.54031772])},
 0.639120922063683)

In [30]:
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

modelo = Pipeline([
    ('scaler', StandardScaler()),
    ('clf', RidgeClassifier())
])
scores = cross_validate(modelo, X, y)
scores, np.mean(scores['test_score'])

({'fit_time': array([3.87529659, 2.5427289 , 2.23372698, 2.51340866, 1.42458844]),
  'score_time': array([0.12741947, 0.18669939, 0.11638117, 0.0910387 , 0.09828711]),
  'test_score': array([0.58947704, 0.76360335, 0.62653827, 0.6756252 , 0.5407308 ])},
 0.6391949311020719)

In [31]:
from sklearn.feature_selection import VarianceThreshold

modelo = Pipeline([
    ('fs', VarianceThreshold(threshold=0.01)),
    ('clf', RidgeClassifier())
])
scores = cross_validate(modelo, X, y)
scores, np.mean(scores['test_score'])

({'fit_time': array([1.49292254, 1.552665  , 1.23112726, 1.25810838, 1.24994898]),
  'score_time': array([0.15799141, 0.06123376, 0.06871104, 0.07850695, 0.07360435]),
  'test_score': array([0.6034784 , 0.77028992, 0.63267414, 0.69466963, 0.53096332])},
 0.6464150810342003)

In [32]:
from sklearn.feature_selection import SelectPercentile, f_classif

modelo = Pipeline([
    ('fsvar', VarianceThreshold()),
    ('fsanova', SelectPercentile(f_classif)),
    ('clf', RidgeClassifier())
])
scores = cross_validate(modelo, X, y)
scores, np.mean(scores['test_score'])

({'fit_time': array([1.64469337, 1.53733993, 1.46387458, 1.71303248, 1.75484276]),
  'score_time': array([0.08056808, 0.08283472, 0.07768226, 0.18807411, 0.08135414]),
  'test_score': array([0.65963013, 0.75532473, 0.67429993, 0.62916301, 0.52558476])},
 0.6488005112564956)

### Testando novas feature selections:


In [33]:
## SelectKBest + f_classif

from sklearn.feature_selection import SelectKBest, f_classif

modelo = Pipeline([
    ('scaler', StandardScaler()),
    ('kbest', SelectKBest(f_classif)),
    ('clf', RidgeClassifier())
])
scores = cross_validate(modelo, X, y)
scores, np.mean(scores['test_score'])

/usr/local/lib/python3.10/dist-packages/sklearn/feature_selection/_univariate_selection.py:112: UserWarning: Features [20] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
/usr/local/lib/python3.10/dist-packages/sklearn/feature_selection/_univariate_selection.py:113: RuntimeWarning: invalid value encountered in divide
  f = msb / msw
/usr/local/lib/python3.10/dist-packages/sklearn/feature_selection/_univariate_selection.py:112: UserWarning: Features [28] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
/usr/local/lib/python3.10/dist-packages/sklearn/feature_selection/_univariate_selection.py:113: RuntimeWarning: invalid value encountered in divide
  f = msb / msw


({'fit_time': array([1.27696514, 1.30453277, 1.28355527, 1.25517821, 1.26157594]),
  'score_time': array([0.09364319, 0.07603741, 0.10318303, 0.08494687, 0.08054829]),
  'test_score': array([0.64184229, 0.75251069, 0.68102959, 0.65092683, 0.50394141])},
 0.6460501637614057)

In [36]:
from sklearn.feature_selection import RFE
from sklearn.svm import SVR

estimator = RidgeClassifier()
selector = RFE(estimator, n_features_to_select=2, step=1)
selector = selector.fit(X, y)

scores = cross_validate(estimator, X[:, selector.support_], y, cv=5)

mean_score = np.mean(scores['test_score'])

print("Scores:", scores)
print("Média das pontuações:", mean_score)

Scores: {'fit_time': array([0.22639298, 0.2801609 , 0.26654387, 0.26627517, 0.36152029]), 'score_time': array([0.02097225, 0.01839447, 0.01665497, 0.02040148, 0.0337317 ]), 'test_score': array([0.5408122 , 0.55140573, 0.54500783, 0.47432058, 0.48759918])}
Média das pontuações: 0.5198291041153699


In [46]:
from sklearn.feature_selection import RFECV
from sklearn.svm import SVR

estimator = RidgeClassifier()
selector = RFECV(estimator, step=1, cv=5)
selector = selector.fit(X, y)

scores = cross_validate(estimator, X[:, selector.support_], y, cv=5)

mean_score = np.mean(scores['test_score'])

print("Scores:", scores)
print("Média das pontuações:", mean_score)

Scores: {'fit_time': array([1.24240255, 1.27537036, 0.9535265 , 0.95170736, 0.96249413]), 'score_time': array([0.06747723, 0.06101418, 0.0389545 , 0.03869724, 0.03802085]), 'test_score': array([0.6031772 , 0.76204573, 0.64613346, 0.67016058, 0.5648698 ])}
Média das pontuações: 0.6492773528460288


In [38]:
# from sklearn.ensemble import RandomForestClassifier

# # ### clf = SVC

# modelo = Pipeline([
#     ('scaler', StandardScaler()),
#     ('clf', RandomForestClassifier())
# ])
# scores = cross_validate(modelo, X, y)
# scores, np.mean(scores['test_score'])

In [39]:
# from sklearn.ensemble import RandomForestClassifier

# # ### clf = SVC

# modelo = Pipeline([
#     ('scaler', StandardScaler()),
#     ('')
#     ('clf', RandomForestClassifier())
# ])
# scores = cross_validate(modelo, X, y)
# scores, np.mean(scores['test_score'])

In [45]:
from sklearn.model_selection import cross_validate
from sklearn.linear_model import RidgeClassifier
from sklearn.pipeline import Pipeline
import numpy as np
from sklearn.svm import LinearSVC

modelo = RidgeClassifier()

pipeline = Pipeline([
    ('feature_selection', SelectFromModel(LinearSVC(penalty="l2", dual=False))),
    ('classifier', modelo)
])

pipeline.fit(X, y)

scores = cross_validate(modelo, X, y)
mean_score = np.mean(scores['test_score'])
print(scores, mean_score)


{'fit_time': array([1.14413095, 0.98501039, 0.96774316, 0.95615458, 0.95546794]), 'score_time': array([0.0715394 , 0.06178498, 0.04956341, 0.04295087, 0.04392362]), 'test_score': array([0.58947704, 0.76359474, 0.62652106, 0.67569405, 0.54031772])} 0.639120922063683


In [44]:
from sklearn.feature_selection import SelectPercentile, mutual_info_classif

modelo = Pipeline([
    ('scaler', StandardScaler()),
    ('fsanova', SelectPercentile(mutual_info_classif)),
    ('clf', RidgeClassifier())
])
scores = cross_validate(modelo, X, y)
scores, np.mean(scores['test_score'])

({'fit_time': array([238.80578828, 231.87446928, 238.42065287, 240.20328999,
         229.21308851]),
  'score_time': array([0.08967304, 0.13907194, 0.06977415, 0.07024455, 0.07021761]),
  'test_score': array([0.57165478, 0.76988546, 0.65883548, 0.64048812, 0.51151443])},
 0.6304756518128163)

#### O melhor ganho no clf Ridge foi com feature selection RFECV, aumentando em aprox. 1% o desempenho do classificador sem feature selection.